# **Ejercicio 4**
##  **Modelos de Regresión** 
Considere el conjunto de datos `Open Problems– Single-Cell Perturbations`. Implemente la versión de regresión de cada uno de los modelos estudiados en clases, esto es KNN y Regresión Lineal en el conjunto de datos suministrado. Construir una tabla de error con las métricas usuales de regresión, MAPE, MAE, RMSE, MSE, R2 (ver Table 2). Utilice la métrica Mean Rowwise Root Mean Squared Error (MRRMSE) en la evaluación y validación, para seleccionar el mejor modelo de regresión.

\[
\begin{array}{|c|c|c|c|c|c|}
\hline
\textbf{Modelo} & \textbf{MAPE} & \textbf{MAE} & \textbf{RMSE} & \textbf{MSE} & \textbf{R2} \\
\hline
\text{K-NN} & \cdots & \cdots & \cdots & \cdots & \cdots \\
\text{Linear Regression} & \cdots & \cdots & \cdots & \cdots & \cdots \\
\hline
\end{array}
\]

\text{Cuadro 2: Modelo de regresión para velocidad del viento.}
